<a href="https://colab.research.google.com/github/sudhir2016/Google-Colab-9/blob/master/Face_Verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an example of using VGGFace for face verification.

It is assumed that directory named test of dataset of photos of 429 individuals with at least one photo each is uploaded in the notebook. I had used subset of LFW dataset.

Pip install VGGFace and MTCNN

In [0]:
pip install keras_vggface

In [0]:
pip install mtcnn


Load various libraries.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions
from mtcnn import MTCNN
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from scipy.spatial.distance import cosine
import cv2
import glob

Following to be used only for extracting from tar file.

In [0]:
#import tarfile

In [0]:
#test=tarfile.open('face1.tgz')

In [0]:
#test.extractall('/content')

Create an empty list for labels and embeddings.

In [0]:
label=[]
emb=[]  

Read folder names as labels and append to label list.

In [0]:
for name in glob.glob('/content/test/*'):
  name1=name[14:]
  label.append(name1)

Print label list and verify its length.

In [0]:
print(label)

In [0]:
len(label)

Define path head for os.path to generate full path for loading images.

In [0]:
a='/content/test'

Define detector as MTCNN and size as 224,224 for resizing image.

In [0]:
detector=MTCNN()
size=(224,224)

Load VGGFace with senet50 with include_top as false to be able to extract embeddings.

In [0]:
vggface = VGGFace(model='senet50',include_top=False, input_shape=(224, 224, 3), pooling='avg')

Now we iteratively find the embeddings of all the 429 images in the dataset and add to the embedding list.

In [0]:
for i in range (429):
  b=label[i]
  c=label[i]+'_0001.jpg'
  d=os.path.join(a,b,c)
  img1=plt.imread(d)  
  out=detector.detect_faces(img1)
  x1, y1, width, height = out[0]['box']
  x2, y2 = x1 + width, y1 + height
  face = img1[y1:y2, x1:x2]
  img2 = Image.fromarray(face)
  img3 = img2.resize(size)
  img4 = np.array(img3)
  img5 = img4.astype('float32')
  img6=np.expand_dims(img5,axis=0)
  img7 = preprocess_input(img6, version=2)
  p=vggface.predict(img7)
  emb.append(p)

Display the last image processed by the model and verify the length of the emdedding list.

In [0]:
plt.imshow(img1)

In [0]:
len(emb)

We now need to identify a test case for our model. We choose Abel Pacheco (Ex president of Costa Rica) because the dataset has four photos for him. So we load and display the original photo of Abel Pacheco that we used to train the model and a new photo of his for testing. 

In [0]:
test_original = plt.imread('/content/test/Abel_Pacheco/Abel_Pacheco_0001.jpg')
plt.imshow(test_original)

In [0]:
test_new = plt.imread('/content/test/Abel_Pacheco/Abel_Pacheco_0002.jpg')
plt.imshow(test_new)

We now run the new photo of Abel Pacheco through the model and obtain embedding.

In [0]:
  out=detector.detect_faces(test_new)
  x1, y1, width, height = out[0]['box']
  x2, y2 = x1 + width, y1 + height
  face = test_new[y1:y2, x1:x2]
  img2 = Image.fromarray(face)
  img3 = img2.resize(size)
  img4 = np.array(img3)
  img5 = img4.astype('float32')
  img6=np.expand_dims(img5,axis=0)
  img7 = preprocess_input(img6, version=2)
  e=vggface.predict(img7)

We now iteratively check the Cosine distance between the embedding of the new photo of Abel Pacheco and embedding of all the images in the dataset. We declare a match in the case where the Cosine distance is found to be less than 0.4. We can see that the model has found the correct match and Cosine distance between original and new photo of Abel Pachico is 0.24.

In [0]:
for i in range (429):
  s=cosine(e,emb[i])
  if s < 0.4 :
    l=label[i]
    print('Match',i,l,s)  
  
